##Mounting drive

In [71]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
# %cd /content/drive/My\ Drive/Data_mining/Lab2/Notebooks
# !ls

#Installing Packages

In [73]:
# !pip install ipynb

##Importing Libraries

In [74]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
import csv
import time
import re
import tracemalloc
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support, pairwise_distances

plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import load_iris, load_digits, load_wine


# Kmedoids Implementation


In [76]:
randomState = 123

In [77]:
def distance(xNum, xNom, xOrd, centerNum, centerNom, centerOrd):
  dist = 0
  if centerNum.shape[0]:
    dist+= np.sum((xNum-centerNum)**2)
  if centerNom.shape[0]:
    dist+= np.sum(xNom!=centerNom)
  if centerOrd.shape[0]:
    dist+=0
  # print('sq dist=',(xNum-centerNum)**2)
  dist = np.sqrt(dist)
  return dist
  
def calculateHammingDistance(X, Y):
    distances = []
    for x in X:
      distance = (x != Y).astype(int).sum(axis = 1) 
      distances.append(distance)

    return np.array(distances)

In [78]:
def assignCluster(XNum, XNom, XOrd, noOfInstances, indOfCenters, k):
  absErr = 0
  withinClusterVar = 0
  clusters= {}
  distMatrix = 0
  centersNum, centersNom, centersOrd = np.array([]), np.array([]), np.array([])
  if XNum.shape[0]:
    centersNum = XNum[indOfCenters]
    distMatrix += (pairwise_distances(XNum, centersNum))**2
  if XNom.shape[0]:
    centersNom = XNom[indOfCenters]
    distMatrix  += calculateHammingDistance(XNom, centersNom)
  if XOrd.shape[0]:
    centersOrd = XOrd[indOfCenters]
  
  absErr = np.sum(np.min(np.sqrt(distMatrix), axis=1))
  withinClusterVar = np.sum(np.min(distMatrix, axis=1))
  clusterLabels = np.argmin(distMatrix, axis=1)


  for i in range(noOfInstances):
    if clusterLabels[i] not in clusters:
      clusters[clusterLabels[i]] = np.array([i])
    else:
      clusters[clusterLabels[i]] = np.append(clusters[clusterLabels[i]], i)
  # print('clusters = ',clusters)
  return clusters, absErr, withinClusterVar

In [89]:
def Kmedoids(XNum, XNom, XOrd, noOfInstances, k):
  eps = 1e-7
  absErr = 0
  indOfCenters = np.random.choice(XNum.shape[0], size=k, replace=False)
  # print('initial indOfCenters=',indOfCenters)
  # print('indOfCenters=',indOfCenters)
  indices = np.arange(noOfInstances)
  availableIdx = np.array([i for i in indices if i not in indOfCenters])

  # assign initial cluster
  clusters, absErr, withinClusterVar = assignCluster(XNum, XNom, XOrd, noOfInstances, indOfCenters, k)
  
  while True:
    updated = 0
    for i in range(indOfCenters.shape[0]):
      np.random.shuffle(availableIdx)
      for j in range(availableIdx.shape[0]):
        prevCenterInd = indOfCenters[i]
        newCenterInd = availableIdx[j]
        indOfCenters[i]= newCenterInd
        availableIdx[j]= prevCenterInd
        newClusters, newAbsErr, newVar = assignCluster(XNum, XNom, XOrd, noOfInstances, indOfCenters, k)
        # print('newAbsErr={}, absErr={}'.format(newAbsErr, absErr))
        if newAbsErr< absErr:
          # print('indOfCenters=',indOfCenters)
          # print('newAbsErr=',newAbsErr)
          absErr= newAbsErr
          withinClusterVar = newVar
          clusters= newClusters
          updated+=1
          break
        else:
          indOfCenters[i]= prevCenterInd
          availableIdx[j]= newCenterInd
    if updated==0:
      break
  
  centers={
      'num': XNum[indOfCenters] if XNum.shape[0] else np.array([]),
      'nom': XNom[indOfCenters] if XNom.shape[0] else np.array([]),
      'ord': XOrd[indOfCenters] if XOrd.shape[0] else np.array([])
  }
  cache={
      'centers': centers,
      'clusters':clusters,
      'absErr': absErr,
      'withinClusterVar' : withinClusterVar
  }
  return cache

## Quality Measures

In [90]:
def calculateBcubedPrecision(clusters, Y):
  bcubedPrecision = 0
  for c in clusters:
    cluster = clusters[c]
    for i in cluster:
      if (cluster.shape[0]-1)!=0:
        bcubedPrecision += len([j for j in cluster if i!=j and Y[i]==Y[j]])/(cluster.shape[0]-1)
  return bcubedPrecision/Y.shape[0]

def calculateBcubedRecall(clusters, Y):
  bcubedRecall = 0
  for c in clusters:
    cluster = clusters[c]
    for i in cluster:
      if Y[Y==Y[i]].shape[0]-1!=0:
        bcubedRecall += len([j for j in cluster if i!=j and Y[i]==Y[j]])/(Y[Y==Y[i]].shape[0]-1)
  return bcubedRecall/Y.shape[0]
def calculateSilhoutteCoeff(XNum, XNom, XOrd, noOfInstances, clusters):
  silhoutteCoeff = 0
  dist={}
  for i in range(noOfInstances):
    dist[i] = np.array([])
    x1Num, x1Nom, x1Ord = np.array([]), np.array([]), np.array([])
    if XNum.shape[0]:
      x1Num = XNum[i]
    if XNom.shape[0]:
      x1Nom = XNom[i]
    if XOrd.shape[0]:
      x1Ord = XOrd[i]
    for j in range(noOfInstances):
      x2Num, x2Nom, x2Ord = np.array([]), np.array([]), np.array([])
      if XNum.shape[0]:
        x2Num = XNum[j]
      if XNom.shape[0]:
        x2Nom = XNom[j]
      if XOrd.shape[0]:
        x2Ord = XOrd[j]
      dist[i]= np.append(dist[i], distance(x1Num, x1Nom, x1Ord, x2Num, x2Nom, x2Ord))
  for c in range(len(clusters)):
    cluster = clusters[c]
    for i in cluster:
      distArray = dist[i]
      a= 0
      b= np.inf
      if (cluster.shape[0]-1)!=0:
        a= np.sum(distArray[cluster])/(cluster.shape[0]-1)
      for j in range(len(clusters)):
        if c!=j:
          newB = np.sum(distArray[clusters[j]])/(clusters[j].shape[0])
          b= min(b, newB)
      silhoutteCoeff+= (b-a)/max(a,b)
  return silhoutteCoeff/noOfInstances

## Run Kmedoids


In [91]:
def runKmedoids(XNum, XNom, XOrd, noOfInstances, noOfIter, k, Y=np.array([]),groundTruth=False):  
  cache = {}
  cache['bcubedPrecision'], cache['bcubedRecall'], cache['silhoutteCoeff'] = None, None, None
  cache['withinClusterVar'] = np.inf
  for i in range(noOfIter):
    np.random.seed(randomState+i)
    # print(np.random.choice(5, 3))
    newCache = Kmedoids(XNum, XNom, XOrd, noOfInstances, k)  
    if cache['withinClusterVar']>newCache['withinClusterVar']:
      cache = newCache
      cache['bcubedPrecision'], cache['bcubedRecall'], cache['silhoutteCoeff'] = None, None, None
  # print('clusters={}\n centers={}\n absErr={} withinClusterVar={}'.format(cache['clusters'], cache['centers'], cache['absErr'], cache['withinClusterVar']))
  # calculating bcubed precision
  if groundTruth:
    cache['bcubedPrecision']  = calculateBcubedPrecision(cache['clusters'], Y)
    cache['bcubedRecall'] = calculateBcubedRecall(cache['clusters'], Y)
  cache['silhoutteCoeff'] = calculateSilhoutteCoeff(XNum, XNom, XOrd, noOfInstances, cache['clusters'])
  # print('bcubedPrecision={} bcubedRecall={} silhoutteCoeff={}\n'.format(cache['bcubedPrecision'], cache['bcubedRecall'], cache['silhoutteCoeff']))
  return cache



### Loading Dataset

In [92]:
# df = pd.read_csv('/content/drive/My Drive/Data_mining/Lab3/Datasets/test_dataset_2.data', sep=",",header=None)
# noOfInstances = df.shape[0]
# print('instances = {}, features= {} '.format(df.shape[0], df.shape[1]))
# numInd = np.arange(df.shape[1])
# dfNum = df.iloc[:,numInd]
# XNum= dfNum.fillna(dfNum.mean()).to_numpy()
# XNum = XNum.astype(float)
# XNum= (XNum-np.min(XNum, axis=0))/(np.max(XNum, axis =0)-np.min(XNum, axis =0))
# Y= np.array([])
# # print(XNum)
# XNom = np.array([])
# XOrd = np.array([])

# import time
# start = time.time()
# cache = runKmedoids(XNum, XNom, XOrd, noOfInstances, noOfIter = 5, k=4, Y=Y, groundTruth= False)
# print(cache)
# end = time.time()
# print('time = ',end-start)

instances = 100, features= 2 
clusters={0: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24]), 3: array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
       42, 43, 44, 45, 46, 47, 48, 49]), 1: array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74]), 2: array([75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91,
       92, 93, 94, 95, 96, 97, 98, 99])}
 centers={'num': array([[0.14028486, 0.17978398],
       [0.65033295, 0.67075249],
       [0.88762563, 0.88392352],
       [0.39115201, 0.34425922]]), 'nom': array([], dtype=float64), 'ord': array([], dtype=float64)}
 absErr=9.61208350362771 withinClusterVar=1.1486144437089243
bcubedPrecision=None bcubedRecall=None silhoutteCoeff=0.5842989482327084

{'centers': {'num': array([[0.14028486, 0.17978398],
       [0.65033295, 0.67075249],
       [0.88762563, 0.88392352],
       [0.391

## Iris

In [93]:
# from sklearn.datasets import load_iris
# X, Y = load_iris(return_X_y= True)
# Y = np.squeeze(Y)
# noOfInstances = X.shape[0]
# print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))
# numInd = np.arange(X.shape[1])
# XNum = X
# XNum = XNum.astype(float)
# XNum= (XNum-np.min(XNum, axis=0))/(np.max(XNum, axis =0)-np.min(XNum, axis =0))
# # print(XNum)
# XNom = np.array([])
# XOrd = np.array([])
# runKmedoids(XNum, XNom, XOrd, noOfInstances, noOfIter = 5, k=3, Y=Y, groundTruth= True)

instances = 150, features= 4 
clusters={0: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), 1: array([ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
        76,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 101, 106, 113,
       119, 121, 123, 126, 127, 133, 134, 138, 142, 146, 149]), 2: array([ 77, 100, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 114,
       115, 116, 117, 118, 120, 122, 124, 125, 128, 129, 130, 131, 132,
       135, 136, 137, 139, 140, 141, 143, 144, 145, 147, 148])}
 centers={'num': array([[0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.47222222, 0.375     , 0.59322034, 0.58333333],
       [0.69444444, 0.41666667, 0.762

{'absErr': 29.713508751451485,
 'bcubedPrecision': 0.8391397849462388,
 'bcubedRecall': 0.8495238095238086,
 'centers': {'nom': array([], dtype=float64),
  'num': array([[0.19444444, 0.58333333, 0.08474576, 0.04166667],
         [0.47222222, 0.375     , 0.59322034, 0.58333333],
         [0.69444444, 0.41666667, 0.76271186, 0.83333333]]),
  'ord': array([], dtype=float64)},
 'clusters': {0: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
  1: array([ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
          63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
          76,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
          90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 101, 106, 113,
         119, 121, 123, 126, 127, 133, 134, 138, 142, 146, 149]),
  2: array([ 77